# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [25]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [26]:
# TODO: Import the necessary libs
# For example: 
import os

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool
from dotenv import load_dotenv
from enum import Enum, auto
from typing import Dict, List


In [27]:
# TODO: Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [28]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
##chroma_client = chromadb.PersistentClient(path="chromadb")
##collection = chroma_client.get_collection(name="udaplay", embedding_function=embedding_fn)

##Retrieve_game tool
def retrieve_game(query: str, n_results: int = 5) -> list[dict]:
    """
    Semantic search: Finds most relevant results in the vector DB.

    Args:
        query: A question or description about games (platforms, names, years, etc.).
        n_results: How many top matches to return.

    Returns:
        A list of dicts, each containing:
            - Platform
            - Name
            - YearOfRelease
            - Description
            - id (the Chroma document id)
            - score (similarity score derived from distance; higher is better)
    """
    if not query or not isinstance(query, str):
        raise ValueError("`query` must be a non-empty string.")

    # Chroma query returns lists grouped by each input query_texts item.
    res = collection.query(
        query_texts=[query],
        n_results=n_results,
        include=["metadatas", "documents", "ids", "distances"]
    )

    # Defensive extraction: Chroma returns nested lists, one per query.
    metadatas = res.get("metadatas", [[]])[0]
    documents = res.get("documents", [[]])[0]
    ids       = res.get("ids", [[]])[0]
    distances = res.get("distances", [[]])[0]

    results = []
    for i in range(len(ids)):
        meta = metadatas[i] if i < len(metadatas) else {}
        doc  = documents[i] if i < len(documents) else ""
        dist = distances[i] if i < len(distances) else None

        # Convert Chroma distance to a 0..1 similarity score (heuristic).
        # Chroma's "distance" is typically cosine distance (0 is identical).
        # We map it to score = 1 - min(max(dist,0),1). If dist > 1, clamp to 0.
        if dist is None:
            score = None
        else:
            score = 1.0 - max(0.0, min(float(dist), 1.0))

        # Normalize metadata keys expected from your add loop
        results.append({
            "Platform":     meta.get("Platform"),
            "Name":         meta.get("Name"),
            "YearOfRelease": meta.get("YearOfRelease"),
            "Description":  meta.get("Description"),
            "id":           ids[i],
            "score":        score,
            # Optional: include the raw document string
            "document":     doc
        })

    return results
    

#### Evaluate Retrieval Tool

In [29]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result

##Evaluate_retrieval:

# tools_evaluate.py

from __future__ import annotations
import json
import os
from typing import List, Dict, Any
from pydantic import BaseModel
from dotenv import load_dotenv
from openai import OpenAI


class EvaluationReport(BaseModel):
    """
    Data class for the LLM judge outcome.
    - useful: whether the documents are useful to answer the question
    - description: detailed explanation supporting the decision
    """
    useful: bool
    description: str


# Initialize environment and client once
load_dotenv('.env')
_OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
_OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL")  # optional (e.g., Azure/OpenAI proxy)
_MODEL_NAME = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

if not _OPENAI_API_KEY:
    # Fail fast with a clear message; Udacity runner will surface this
    raise RuntimeError("OPENAI_API_KEY is missing in .env")

_client = OpenAI(api_key=_OPENAI_API_KEY, base_url=_OPENAI_BASE_URL)


def evaluate_retrieval(
    question: str,
    retrieved_docs: List[Dict[str, Any]],
    max_docs: int = 8,
) -> EvaluationReport:
    """
    Tool: evaluate_retrieval
    ------------------------
    Based on the user's question and on the list of retrieved documents,
    it will analyze the usability of the documents to respond to that question.

    Args:
        - question: original question from user
        - retrieved_docs: retrieved documents most similar to the user query in the Vector Database

    Returns:
        EvaluationReport:
            - useful: whether the documents are useful to answer the question
            - description: description about the evaluation result
    """
    # Basic validation
    if not isinstance(question, str) or not question.strip():
        return EvaluationReport(useful=False, description="Invalid question.")
    if not retrieved_docs:
        return EvaluationReport(useful=False, description="No documents retrieved to evaluate.")

    # Compact, LLM-friendly view of docs (limit to max_docs, truncate long descriptions)
    lines: List[str] = []
    for i, d in enumerate(retrieved_docs[:max_docs], start=1):
        name = str(d.get("Name", "Unknown"))
        plat = str(d.get("Platform", ""))
        year = str(d.get("YearOfRelease", ""))
        desc = str(d.get("Description", ""))[:500]  # keep prompt size reasonable
        score = d.get("score")
        doc_id = d.get("id")
        lines.append(
            f"Doc {i}: Name={name}; Platform={plat}; Year={year}; Score={score}; Id={doc_id}; Description={desc}"
        )
    docs_block = "\n".join(lines)

    # LLM judge prompt per TODO
    prompt = f"""
You are an expert evaluator.
Your task is to evaluate if the provided documents are enough to respond to the query.

Query:
\"\"\"{question}\"\"\"

Documents:
{docs_block}

Instructions:
- Determine if the documents, as a set, are sufficient and relevant to answer the query.
- Consider coverage of key facts the query implies (e.g., developer, release date, platform), when applicable.
- If not sufficient, explain what's missing or ambiguous.
- Give a detailed explanation, so it's possible to take an action to accept it or not.

Respond ONLY in strict JSON with the following keys:
- "useful": true or false
- "description": a concise but informative explanation
"""

    # Call the model; enforce JSON output if supported by your SDK version
    try:
        response = _client.responses.create(
            model=_MODEL_NAME,
            input=prompt,
            response_format={"type": "json_object"},  # helps ensure valid JSON
        )
        raw_text = response.output_text.strip()
    except Exception as e:
        return EvaluationReport(
            useful=False,
            description=f"LLM evaluation failed: {e}"
        )

    # Parse JSON safely
    try:
        parsed = json.loads(raw_text)
        useful = bool(parsed.get("useful"))
        description = str(parsed.get("description", "")).strip() or "No description provided."
        return EvaluationReport(useful=useful, description=description)
    except Exception:
        # Fallback if model returned non-JSON
        return EvaluationReport(
            useful=False,
            description=f"LLM response could not be parsed as JSON: {raw_text}"
        )



#### Game Web Search Tool

In [44]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. 
query = "in what year was need for speed first sold and by whom?"

##Game web Search - External search helper (graceful fallback)

def game_web_search(query: str, max_results: int = 5) -> list[dict]:
    """
    Lightweight web search helper for game-related queries.
    Tries to fetch simple web results. Falls back gracefully if no internet.

    Args:
        query: Search query (e.g., game name + platform + release year).
        max_results: Max items to return.

    Returns:
        A list of dicts: { "title": str, "url": str, "snippet": str }.
        If web access is unavailable, returns an empty list and a note in 'snippet'.
    """
    import os



    try:
        import requests
        from bs4 import BeautifulSoup  # requires 'beautifulsoup4' installed
    except Exception:
        # Fallback (no web libs)
        return [{
            "title": "Web search unavailable",
            "url": "",
            "snippet": "Requests/BeautifulSoup not available in this environment."
        }]

    try:
        # Very simple HTML search using DuckDuckGo (no API key)
        resp = requests.get("https://duckduckgo.com/html/", params={"q": query}, timeout=8)
        if resp.status_code != 200:
            return [{
                "title": "Web search failed",
                "url": "",
                "snippet": f"HTTP {resp.status_code} while searching for '{query}'."
            }]
        soup = BeautifulSoup(resp.text, "html.parser")
        results = []
        for a in soup.select(".result__a")[:max_results]:
            title = a.get_text(strip=True)
            url = a.get("href", "")
            snippet_tag = a.find_parent("div", class_="result").select_one(".result__snippet")
            snippet = snippet_tag.get_text(strip=True) if snippet_tag else ""
            results.append({"title": title, "url": url, "snippet": snippet})
        if not results:
            results.append({"title": "No results parsed", "url": "", "snippet": "Parsing returned no items."})
        return results
    except Exception as e:
        return [{
            "title": "Web search error",
            "url": "",
            "snippet": f"Exception during web search: {e}"
        }]





# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game

In [ ]:

# test_evaluate.py
from tools_retrieve import retrieve_game
from tools_evaluate import evaluate_retrieval

query = 'Who developed "FIFA 21"?'
results = retrieve_game(query, n_results=5)

print("\nRetrieved Results:")
for r in results:
    print(f"{r['Name']} | score={r['score']} | Year={r['YearOfRelease']}")

print("\nEvaluation Metrics:")
metrics = evaluate_retrieval(query, results)
for k, v in metrics.items():
    print(f"{k}: {v}")


### Agent

In [46]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed


class AgentState(Enum):
    ASK = auto()
    RAG = auto()
    EVAL = auto()
    WEB = auto()
    PARSE = auto()
    STORE = auto()
    REPORT = auto()

class UdaPlayAgent:
    def __init__(self): pass

    def run(self, question: str) -> Dict[str, str]:
        state = AgentState.ASK
        rag_hits: List[RetrievalHit] = []
        web_records: List[GameRecord] = []
        confidence = 0.0
        metrics = {}
        resolved: Dict[str, str] = {}
        reasoning_steps: List[str] = []

        # RAG
        state = AgentState.RAG
        rag_hits = retrieve_game(question)
        reasoning_steps.append(f"RAG returned {len(rag_hits)} hits.")

        # Evaluate
        state = AgentState.EVAL
        confidence, metrics = evaluate_retrieval(question, rag_hits)
        reasoning_steps.append(f"Evaluation metrics: {metrics} → confidence={confidence:.3f}.")

        # Decide fallback
        if confidence < MIN_CONFIDENCE:
            state = AgentState.WEB
            web_records = game_web_search(question)
            reasoning_steps.append(f"Fallback to web produced {len(web_records)} candidates.")
            state = AgentState.PARSE
            # prefer first web record; refine resolved fields
            best_web = web_records[0] if web_records else None
            if best_web:
                resolved = {
                    "title": best_web.title or _infer_title(question),
                    "developer": best_web.developer or "",
                    "publisher": best_web.publisher or "",
                    "release_date": best_web.release_date or "",
                    "platforms": ", ".join(best_web.platforms) if best_web.platforms else ""
                }
                # store memory
                state = AgentState.STORE
                added = persist_new_knowledge(web_records[:3])
                reasoning_steps.append(f"Persisted {added} new web‑sourced records.")
                # recompute confidence (boost slightly due to fresh authoritative source)
                confidence = min(1.0, max(confidence, 0.72))
        else:
            # Resolve from local hits (majority vote on top 3)
            resolved = _resolve_from_local(question, rag_hits[:3])
            reasoning_steps.append("Resolved facts from local dataset.")

        state = AgentState.REPORT
        from report import render
        report = build_report(
            question=question,
            resolved=resolved,
            confidence=confidence,
            sources_local=rag_hits,
            sources_web=web_records,
            reasoning=" → ".join(reasoning_steps)
        )
        return {"markdown": render(report)}

def _infer_title(question: str) -> str:
    import re
    m = re.search(r"“([^”]+)”|\"([^\"]+)\"", question)
    return m.group(1) or m.group(2) if m else ""

def _resolve_from_local(question: str, hits: List[RetrievalHit]) -> Dict[str, str]:
    # choose best hit and pull structured fields (simple heuristic: highest score)
    if not hits: return {}
    best = sorted(hits, key=lambda h: h.score, reverse=True)[0]
    resolved = {
        "title": best.title,
        "developer": best.record.developer or "",
        "publisher": best.record.publisher or "",
        "release_date": best.record.release_date or "",
        "platforms": ", ".join(best.record.platforms) if best.record.platforms else ""
    }
    return resolved

    
!pwd
!ls



2431.55s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/workspace/Code/project/starter


2437.15s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


README.md			  __pycache__  games	  test_evaluate_llm.py
Udaplay_01_starter_project.ipynb  agent.py     lib	  test_tools.py
Udaplay_02_starter_project.ipynb  chromadb     models.py  tools.py


In [47]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?

# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X released for Playstation 5?

from IPython.display import Markdown, display
from agent import UdaPlayAgent  # Ensure agent.py contains the UdaPlayAgent class

# Instantiate the agent
agent = UdaPlayAgent()

# Questions to ask
questions = [
    "When Pokémon Gold and Silver was released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X released for Playstation 5?"
]

print("### UdaPlay Agent Responses ###\n")
for q in questions:
    result = agent.run(q)  # agent.run returns {"markdown": render(report)}
    display(Markdown(result["markdown"]))


ModuleNotFoundError: No module named 'search'

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes